# Data Cleanup

In [82]:
# importing the libraries
from utils.DataIngestion import get_config, print_config, get_path, ingest_data
from utils.HelperFunctions import negative_check, not_in_list, out_of_bounding_box, out_of_range

import numpy as np
import pandas as pd
import os
import logging
import numbers

### Loading Data and Configurations

In [83]:
# let's load the dataframe and configuration files
path = get_path()
print(f"Path is {path}")

config = get_config("data_preparation_config.yml")
print_config(config)

Path is D:\Machine Learning\MACHINE LEARNING PROJECTS\airbnb_price_prediction\data
Path, path_to_yaml: D:\Machine Learning\MACHINE LEARNING PROJECTS\airbnb_price_prediction\notebooks\data_preparation_config.yml


Config Value general --> {'load_from_scratch': False, 'save_raw_dataframe': False, 'save_transformed_dataframe': True, 'remove_bad_values': True}
Config Value columns --> {'categorical': ['neighbourhood_group', 'room_type'], 'continuous': ['minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'latitude', 'longitude'], 'date': ['last_review'], 'text': ['name', 'host_name'], 'excluded': ['price', 'id']}
Config Value category_defaults --> {'categorical': 'missing', 'continuous': 0.0, 'text': 'missing', 'date': datetime.date(2019, 1, 1), 'excluded': 'missing'}
Config Value category_invalid_replacements --> {'categorical': 'bad_categorical', 'continuous': 'bad_continuous', 'text': 'bad_text', 'date': 'bad_date', 'excluded': 'bad_excluded'}
Co

In [84]:
# loading the data
df = ingest_data(path=path,
                input_csv=config['file_names']['input_csv'],
                pickled_input_dataframe=config['file_names']['pickled_input_dataframe'],
                save_raw_dataframe=config['general']['save_raw_dataframe'],
                load_from_scratch=config['general']['load_from_scratch'])
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

### General Cleaning 

#### Replacing Numeric and Categorical Variables

In [86]:
def fill_missing(df, columns, defaults):
    ''' replace missing values with placeholders by column type
    Args:
        df: dataframe to process for missing values
        columns: dictionary of columns with the column type (key = column type, value = column name)
        defaults: dictionary of replacements of missing values by column type
    Returns:
        dataset: dataframe with missing values replaced with default values
    '''
    logging.debug("Before Missing Values")
    for col_cat in columns:
        print(f"Column Category is {col_cat}")
        for col in columns[col_cat]:
            print(f"Filling missing values in {col} with deafult {defaults[col_cat]}")
            df[col].fillna(defaults[col_cat], inplace=True)
            print(f"Missing values in {col}: {str(df[col].isna().sum())}")
            
    return df

In [87]:
def replace_if_not_in_list(x, replace_x, lst):
    ''' check if a value is in a list
    Args:
        x: value to check
        replace_x: replacement value
        lst: list in which to check for the value
    Returns: 
        x if value is in not in list, replace_x otherwise
    '''
    if x in lst:
        return x
    else:
        return replace_x

In [88]:
def replace_if_non_numeric(x, replace_x):
    ''' check if a value is non-numeric and replace if so
    Args:
        x: value to check
        replace_x: replacement value
    Returns: 
        x if value is numeric, replace_x otherwise
    '''
    if isinstance(x, numbers.Number):
        return x
    else:
        return replace_x

In [89]:
def replace_if_negative(x, replace_x):
    ''' checks if a value is negative if so replaces it
    Args:
        x: value to check
        replace_x: replacement value
    Returns: 
        x if value is not negative, replace_x otherwise
    '''
    if x >= 0:
        return x
    else:
        return replace_x

In [90]:
# main function to replace invalid values
def replace_invalid_values(df, columns, valid_values, invalid_value_replacements, non_neg_continuous):
    ''' replace invalid with placeholders
    Args:
        df: dataframe for assessment
        columns: dictionary of column names by category
        valid_values: dictionary of valid values for categorical columns
        invalid_value_replacements: dictionary of replacement values
        non_neg_continuous: list of continuous columns with only non-negative values as valid
    Returns:
        df: updated dataframe
    '''
    # categorical columns 
    for col in columns['categorical']:
        print(f"Non valid values in {col}: {df[col].apply(lambda x: not_in_list(x, valid_values[col])).sum()}")
        df[col] = df[col].apply(lambda x: replace_if_not_in_list(x, invalid_value_replacements['categorical'], valid_values[col]))
        
    # continuous columns
    for col in columns['continuous']:
        # replace non-numeric values
        df[col] = df[col].apply(lambda x: replace_if_non_numeric(x, invalid_value_replacements['continuous']))
        # replace if negative
        if col in non_neg_continuous:
            df[col] = df[col].apply(lambda x: replace_if_negative(x, invalid_value_replacements['continuous']))
            
    return df

#### Replacing Geospatial Variables

In [91]:
def replace_if_outside_bounding_box(latitude, longitude, replace_lat, replace_long, bounding_box):
    ''' check whether longitude and latitude values are outside bounding box
    Args:
        latitude: latitude portion of location
        longitude: longitude portion of location
        replace_lat: replacement value for latitude
        replace_long: replacement value for longitude
        bounding_box: dictionary with max and min value to compare to.
    Returns:
        latitude, longitude: 1 if out of range, 0 otherwise
    '''
    if (latitude > bounding_box['max_lat'] or latitude < bounding_box['min_lat']) \
    or (longitude > bounding_box['max_long'] or longitude < bounding_box['min_long']):
        latitude = replace_lat
        longitude = replace_long
    return (latitude, longitude)

In [92]:
def replace_time(date_time_value, time_value):
    ''' 
    given a dataframe, replace the time portion
    '''
    date_time_value = date_time_value.replace(hour=time_value.hour, minute=time_value.minute, second_time=time_value.second)
    return date_time_value

In [93]:
# the main function to replace geospatial data
def geo_replacement(df, replace_lat, replace_long, bounding_box):
    ''' assess the geospatial data by checking values of longitude and latitude
    Args:
        df: dataframe to assess
        replace_lat: replacement for latitude value
        replace_long: replacement for longitude value
        bounding_box: dictionary of max and min values of bounding box
    Returns:
        df: updated dataframe
    '''
    df['latitude', 'longitude'] = df.apply(lambda x: replace_if_outside_bounding_box(x.latitude, x.longitude, replace_lat, replace_long, bounding_box), axis=1)
    return df


### Master Cell

In [94]:
print(f"Columns: {config['columns']}")
print("-" * 80)
print(f"Category defaults: {config['category_defaults']}")

Columns: {'categorical': ['neighbourhood_group', 'room_type'], 'continuous': ['minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'latitude', 'longitude'], 'date': ['last_review'], 'text': ['name', 'host_name'], 'excluded': ['price', 'id']}
--------------------------------------------------------------------------------
Category defaults: {'categorical': 'missing', 'continuous': 0.0, 'text': 'missing', 'date': datetime.date(2019, 1, 1), 'excluded': 'missing'}


In [95]:
# filling missing values
df = fill_missing(df, config['columns'], config['category_defaults'])
df.info()

Column Category is categorical
Filling missing values in neighbourhood_group with deafult missing
Missing values in neighbourhood_group: 0
Filling missing values in room_type with deafult missing
Missing values in room_type: 0
Column Category is continuous
Filling missing values in minimum_nights with deafult 0.0
Missing values in minimum_nights: 0
Filling missing values in number_of_reviews with deafult 0.0
Missing values in number_of_reviews: 0
Filling missing values in reviews_per_month with deafult 0.0
Missing values in reviews_per_month: 0
Filling missing values in calculated_host_listings_count with deafult 0.0
Missing values in calculated_host_listings_count: 0
Filling missing values in latitude with deafult 0.0
Missing values in latitude: 0
Filling missing values in longitude with deafult 0.0
Missing values in longitude: 0
Column Category is date
Filling missing values in last_review with deafult 2019-01-01
Missing values in last_review: 0
Column Category is text
Filling missin

In [96]:
# replacing invalid values
df = replace_invalid_values(df=df,
                           columns=config['columns'],
                           valid_values=config['valid_values'],
                           invalid_value_replacements=config['category_invalid_replacements'],
                           non_neg_continuous=config['non_negative_continuous'])

Non valid values in neighbourhood_group: 0
Non valid values in room_type: 0


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48895 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48895 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [98]:
# replacing geospatial data
df = geo_replacement(df, config['latitude_replacement'], config['longitude_replacement'], config['bounding_box'])

In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48895 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48895 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

##### Saving transformed data:

In [100]:
if config['general']['save_transformed_dataframe']:
    print("Path:", path)
    filename = os.path.join(path, config['file_names']['pickled_output_dataframe'])
    print("File Name:", filename)
    df.to_pickle(filename)

Path: D:\Machine Learning\MACHINE LEARNING PROJECTS\airbnb_price_prediction\data
File Name: D:\Machine Learning\MACHINE LEARNING PROJECTS\airbnb_price_prediction\data\AB_NYC_2019_output_nov8.pkl


In [103]:
# loading the pickled output file
pickled_output_file = pd.read_pickle(os.path.join(path, config['file_names']['pickled_output_dataframe']))
pickled_output_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48895 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48895 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     